In [1]:
import pandas as pd
import sqlite3

In [14]:
df = pd.read_csv('academy_awards.csv', encoding='ISO-8859-1')
df.head()

Year               Category          Nominee  \
0  2010 (83rd)  Actor -- Leading Role    Javier Bardem   
1  2010 (83rd)  Actor -- Leading Role     Jeff Bridges   
2  2010 (83rd)  Actor -- Leading Role  Jesse Eisenberg   
3  2010 (83rd)  Actor -- Leading Role      Colin Firth   
4  2010 (83rd)  Actor -- Leading Role     James Franco   

                          Additional Info Won? Unnamed: 5 Unnamed: 6  \
0                      Biutiful {'Uxbal'}   NO        NaN        NaN   
1           True Grit {'Rooster Cogburn'}   NO        NaN        NaN   
2  The Social Network {'Mark Zuckerberg'}   NO        NaN        NaN   
3    The King's Speech {'King George VI'}  YES        NaN        NaN   
4              127 Hours {'Aron Ralston'}   NO        NaN        NaN   

  Unnamed: 7 Unnamed: 8 Unnamed: 9 Unnamed: 10  
0        NaN        NaN        NaN         NaN  
1        NaN        NaN        NaN         NaN  
2        NaN        NaN        NaN         NaN  
3        NaN        NaN        NaN         NaN  
4        NaN        NaN        NaN         NaN

In [15]:
# clean Year, extract first 4 int and convert type
df['Year'] = df['Year'].str[0:4]
df['Year'] = df['Year'].astype('int64')
# filter the row after 2000
later_than_2000 = df[df['Year'] > 2000]
# filter the interested row
award_categories = ["Actor -- Leading Role","Actor -- Supporting Role", "Actress -- Leading Role", "Actress -- Supporting Role"]
nominations = later_than_2000[later_than_2000["Category"].isin(award_categories)]
nominations.head()


Year               Category          Nominee  \
0  2010  Actor -- Leading Role    Javier Bardem   
1  2010  Actor -- Leading Role     Jeff Bridges   
2  2010  Actor -- Leading Role  Jesse Eisenberg   
3  2010  Actor -- Leading Role      Colin Firth   
4  2010  Actor -- Leading Role     James Franco   

                          Additional Info Won? Unnamed: 5 Unnamed: 6  \
0                      Biutiful {'Uxbal'}   NO        NaN        NaN   
1           True Grit {'Rooster Cogburn'}   NO        NaN        NaN   
2  The Social Network {'Mark Zuckerberg'}   NO        NaN        NaN   
3    The King's Speech {'King George VI'}  YES        NaN        NaN   
4              127 Hours {'Aron Ralston'}   NO        NaN        NaN   

  Unnamed: 7 Unnamed: 8 Unnamed: 9 Unnamed: 10  
0        NaN        NaN        NaN         NaN  
1        NaN        NaN        NaN         NaN  
2        NaN        NaN        NaN         NaN  
3        NaN        NaN        NaN         NaN  
4        NaN        NaN        NaN         NaN

In [16]:
nominations.shape

(200, 11)

In [17]:
# clean Won? column replace all NO values with 0 and all YES values with 1
replacements = { "NO": 0, "YES": 1 }
nominations["Won?"] = nominations["Won?"].map(replacements)
nominations["Won"] = nominations["Won?"]
nominations.head()

/dataquest/system/env/python3/lib/python3.4/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/dataquest/system/env/python3/lib/python3.4/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Year               Category          Nominee  \
0  2010  Actor -- Leading Role    Javier Bardem   
1  2010  Actor -- Leading Role     Jeff Bridges   
2  2010  Actor -- Leading Role  Jesse Eisenberg   
3  2010  Actor -- Leading Role      Colin Firth   
4  2010  Actor -- Leading Role     James Franco   

                          Additional Info  Won? Unnamed: 5 Unnamed: 6  \
0                      Biutiful {'Uxbal'}     0        NaN        NaN   
1           True Grit {'Rooster Cogburn'}     0        NaN        NaN   
2  The Social Network {'Mark Zuckerberg'}     0        NaN        NaN   
3    The King's Speech {'King George VI'}     1        NaN        NaN   
4              127 Hours {'Aron Ralston'}     0        NaN        NaN   

  Unnamed: 7 Unnamed: 8 Unnamed: 9 Unnamed: 10  Won  
0        NaN        NaN        NaN         NaN    0  
1        NaN        NaN        NaN         NaN    0  
2        NaN        NaN        NaN         NaN    0  
3        NaN        NaN        NaN         NaN    1  
4        NaN        NaN        NaN         NaN    0

In [18]:
# drop useless columns
drop_cols = ["Won?","Unnamed: 5", "Unnamed: 6","Unnamed: 7", "Unnamed: 8", "Unnamed: 9", "Unnamed: 10"]
final_nominations = nominations.drop(drop_cols, axis=1)
final_nominations.head()

Year               Category          Nominee  \
0  2010  Actor -- Leading Role    Javier Bardem   
1  2010  Actor -- Leading Role     Jeff Bridges   
2  2010  Actor -- Leading Role  Jesse Eisenberg   
3  2010  Actor -- Leading Role      Colin Firth   
4  2010  Actor -- Leading Role     James Franco   

                          Additional Info  Won  
0                      Biutiful {'Uxbal'}    0  
1           True Grit {'Rooster Cogburn'}    0  
2  The Social Network {'Mark Zuckerberg'}    0  
3    The King's Speech {'King George VI'}    1  
4              127 Hours {'Aron Ralston'}    0

In [19]:
# Cleaning up the Additional Info column
additional_info_one = final_nominations["Additional Info"].str.rstrip("'}")
additional_info_two = additional_info_one.str.split(" {'")
movie_names = additional_info_two.str[0]
characters = additional_info_two.str[1]
final_nominations["Movie"] = movie_names
final_nominations["Character"] = characters
final_nominations = final_nominations.drop("Additional Info", axis=1)
final_nominations.head()

Year               Category          Nominee  Won               Movie  \
0  2010  Actor -- Leading Role    Javier Bardem    0            Biutiful   
1  2010  Actor -- Leading Role     Jeff Bridges    0           True Grit   
2  2010  Actor -- Leading Role  Jesse Eisenberg    0  The Social Network   
3  2010  Actor -- Leading Role      Colin Firth    1   The King's Speech   
4  2010  Actor -- Leading Role     James Franco    0           127 Hours   

         Character  
0            Uxbal  
1  Rooster Cogburn  
2  Mark Zuckerberg  
3   King George VI  
4     Aron Ralston

In [20]:
final_nominations.dtypes

Year          int64
Category     object
Nominee      object
Won           int64
Movie        object
Character    object
dtype: object

In [21]:
conn = sqlite3.connect('nominations.db')
final_nominations.to_sql('nominations', conn, index=False)

In [22]:
conn.execute('pragma table_info(nominations);').fetchall()

[(0, 'Year', 'INTEGER', 0, None, 0),
 (1, 'Category', 'TEXT', 0, None, 0),
 (2, 'Nominee', 'TEXT', 0, None, 0),
 (3, 'Won', 'INTEGER', 0, None, 0),
 (4, 'Movie', 'TEXT', 0, None, 0),
 (5, 'Character', 'TEXT', 0, None, 0)]